<div style="padding: 10px; background-color: #ecf0f1;">
**<span style="color:red">NOTE:</span>** This notebook requires [`BSpline2.scala`](http://cobweb.cs.uga.edu/~mec/BSpline2.scala) to be placed in the root of your JupyterHub workspace. If you're on `hub.aods.io`, the you can do this using the following AODS Upload link: http://upload.aods.io/http://cobweb.cs.uga.edu/~mec/BSpline2.scala.
</div>

<div style="padding: 10px; background-color: #ecf0f1;">
**<span style="color:red">NOTE:</span>** Commands that begin with a single colon (e.g., <code>:silent</code>) are [Scala REPL commands](https://docs.scala-lang.org/overviews/repl/overview.html). Commands that begin with a double colon (e.g., <code>::relation</code>) are [ScalaTion Kernel commands](https://github.com/scalation/scalation_kernel/blob/master/USER.md). Anything else in a code cell is assumed to be Scala code. 
</div>

# Regression Splines
Suppose you want to approximate the function $x(t)$ for some process that you have sampled over the input domain (usually time). If the process isn't linear, then directly using multiple linear regression may not be a good idea. Instead, you can better model the curvature of the process using a regression spline.

With a regression spline, the function is modeled as a linear combination of basis functions:

$$ f(t) = \sum_i c_i \phi_i(t) . $$

While many different types of basis functions exist (e.g., polynomial, Fourier, etc.), it is common to use cubic [B-spline](https://en.wikipedia.org/wiki/B-spline#Definition) basis functions due to their appealing geometric properties, as discussed in [Patrikalakis & Maekawa (2009)](http://web.mit.edu/hyperbook/Patrikalakis-Maekawa-Cho/node17.html). The remainder of this notebook demonstrates how to reformulate a regression spline model as a multiple linear regression in order to approximate a function for the number (in thousands) of Australian residents measured quarterly from March 1971 to March 1994. To do this, we are going use the [Quarterly Time Series of the Number of Australian Residents](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/austres.html) dataset from the R [`datasets`](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/00Index.html) package. For convenience, a copy of this dataset is provided at http://cobweb.cs.uga.edu/~mec/austres.csv.

First, let's import some packages from ScalaTion:

In [ ]:
:silent                        // suppress extra output
import scalation.analytics._   // for regression
import scalation.linalgebra._  // for linear algebra support
import scalation.columnar_db._ // for relational algebra support
:paste -raw BSpline2.scala

Next, let's load in the dataset using a [`Relation`](http://cobweb.cs.uga.edu/~jam/scalation_1.3/scalation_mathstat/target/scala-2.12/api/scalation/relalgebra/Relation$.html) to see what's available:

In [ ]:
val url = "http://cobweb.cs.uga.edu/~mec/austres.csv" // dataset url
val rel = Relation(url, "austres", "SDD", 0, ",")     // create relation
::relation rel

It looks like we want to use the second column (index 1) to represent points in the input domain and the last column (index 2) to represent our response. Let's extract them as vectors:

In [ ]:
val t = rel.toVectorD(1) // input / time point vector
val y = rel.toVectorD(2) // response vector
::plotv y

## Generating B-spline Basis Functions
Now, let's prepare for the creation of our B-spline basis functions. The degree of a B-spline basis function is one less than its order. Therefore, we'll use $k=4$ to denote the order for a cubic B-spline. Furthermore, B-spline basis functions are parameterized according to a vector of non-decreasing "knots" across the input domain. The regression spline will only be defined on the closed interval defined by the first and last elements of the knot vector. As described in [Patrikalakis, N. M., & Maekawa, T. (2009)](http://web.mit.edu/hyperbook/Patrikalakis-Maekawa-Cho/node17.html), it is often desirable to "clamp" the knot vector by repeating the first and last time points $k$ times to ensure that the resulting regression spline passes through the first and last points in the response. **<span style="color:blue">NOTE:</span>** We encourage the user to try the remaining steps in this notebook with different sets of internal knots (keeping the endpoints the same).

In [ ]:
val k     = 4                        // order 4; degree 3
val knots = BSpline2Util.clamp(k, t) // clamped knot vector
val bs    = new BSpline2(knots, k)   // cubic b-spline basis generator

Now we are ready to define $\phi_i(t)$:

In [ ]:
val nsf  = bs.count()                     // number of spline basis functions (for i)
def phi (i: Int)(t: Double) = bs(k)(i)(t) // the basis function
val step = 5E-4                           // step size for plotting 

Let's take a look at the first five B-spline basis functions along the sub-interval defined by $[t_0, t_5]$:

In [ ]:
def bf(i: Int) = VectorD((t(0) to t(5) by step).map(phi(i)).toSeq) // evaluate basis function i along the interval
val bfs = MatrixD((0 until 5).map(bf).toSeq)                       // create matrix with a column for each function
::plotm bfs

As you can see, the first two basis functions (and the last two, although unseen) seem exhibit more acceleration than the rest. This is verified by looking at their second derivatives along the same subinterval:

In [ ]:
def d2phi(i: Int)(t: Double) = bs.derivative(k, 2, t)(i)
def d2bf(i: Int) = VectorD((t(0) to t(5) by step).map(d2phi(i)).toSeq)
val d2bfs = MatrixD((0 until 5).map(d2bf).toSeq)
::plotm d2bfs

Let's keep thos accerlation in mind for later. It's going to have an impact on the regression.

## Ordinary Least Squares Method 
To reformulate the regression spline model as a multiple linear regression model, we need to construct a design matrix using the basis functions. Each column in the matrix will represent one of the basis functions. Each row will represent each of the basis functions evaluated at a particular input point in the closed interval defined by the knot vector. Then, we have:

$$ f(\mathbf{t}) = X\mathbf{c}, $$

where $\mathbf{t}$ is the vector of input points and $\mathbf{c}$ is a coefficient vector to be estimated. Now suppose we have a set of samples for $f(t)$ over the input points:

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e}, $$

where $\mathbf{y}$ is the response vector and $\mathbf{e}$ is the error not explained by the model. This allows us to derive a formulation for $\mathbf{e}$ in terms of $X$ and $\mathbf{c}$:

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e}, $$

$$ \mathbf{y} = f(\mathbf{t}) + \mathbf{e} = X\mathbf{c} + \mathbf{e}, $$

$$ \mathbf{y} = X\mathbf{c} + \mathbf{e}, $$

$$ \mathbf{e} = \mathbf{y} - X\mathbf{c}. $$

Now that we have a formulation for the error, let's find an estimate for $\mathbf{c}$ that minimizes the *sum of squared error*  (i.e., $\mathbf{e}'\mathbf{e}$). This can be solved using the ordinary least squares (OLS) method:

$$ D^1_\mathbf{c}[\mathbf{e}'\mathbf{e} ] = 0$$

$$ D^1_\mathbf{c}[(\mathbf{y} - X\mathbf{c})'(\mathbf{y} - X\mathbf{c}) ] = 0$$

$$ -2X'\mathbf{y} + 2X'X\mathbf{c} = 0$$

We can divide both sides by two:

$$ -X'\mathbf{y} + X'X\mathbf{c} = 0$$

$$ X'X\mathbf{c} =  X'\mathbf{y}$$

$$ X'X \mathbf{c} =  X'\mathbf{y}$$

$$ \hat{\mathbf{c}} = (X'X)^{-1}X'\mathbf{y} $$

Let's create the design matrix and solve for the coefficient vector using the OLS method:

In [ ]:
val X = new MatrixD (t.dim, bs.count())                   // create empty design matrix
for (j <- X.range1; i <- X.range2) X(j, i) = phi(i)(t(j)) // evaluate each spline at each input point
//val X = bs(k, t) // pending bug fix from Dr. Miller
val c = (X.t * X).inverse * X.t * y                       // solve for x using OLS

Before we define $f(t)$ using the regression spline, let's look at the fit of the multiple linear regression model:

In [ ]:
val z   = X * c   // predicted response
val e   = y - z   // residual
val sse = e dot e // sum of squared error
::plotv y z
println(s"SSE = $sse")

Oh no! What happened? The sum of squared error (SSE) is huge. It looks like most of the fit is good, including the end points (as discussed above, due to clamping the knot vector), however, the fit along the subintervals approaching each end point appear to be extremely poor. This is due to the acceleration of the first two and last two basis functions that we observed. The solution? Let's fit the model in such a way as to penalize lots of acceleration. To do this, we use a technique called *smoothing*!

## Smoothing Splines

Roughly speaking, a smoothing spline is a regularized regression spline with a penalty based on the second derivative of the basis functions. If the current sequence of basis functions is used to approximate the position of the function at each input, then the second derivative should indicate how fast the curve is accellerating.

Instead of simply minimizing $\mathbf{e}'\mathbf{e}$ like we did before, we'll minimize $\mathbf{e}'\mathbf{e} + \lambda \lvert\lvert \mathbf{c} \cdot D^2\phi  \rvert\rvert^2_2$. If we recall that the predicted response is estimated by $X \mathbf{c}$, then it's clear that this criterion explodes whenever the basis functions exhibit a large amount of acceleration. 

While criterion certainly looks nice in terms of expression, it easier to work with $D^2\phi$ in matrix form. To do this, construct a matrix such that each column represents the second derivative of a basis function and each row represents the evaluation of the second derivatives at a particular input point. Once we have that matrix, we can square it to get the norm. Let $P$ represent this norm.

In [ ]:
val d2bs = new MatrixD(X.dim1, X.dim2)
for (j <- d2bs.range1) d2bs(j) = bs.derivative(k, 2, t(j))
val P    = d2bs.t * d2bs

Now we are ready to find an estimate for $\mathbf{c}$ that minimizes the *penalized sum of squared error* criterion (i.e., $\mathbf{e}'\mathbf{e} + \lambda \lvert\lvert \mathbf{c} \cdot D^2\phi  \rvert\rvert^2_2 = \mathbf{e}'\mathbf{e} + \lambda \mathbf{c}' P \mathbf{c}$. This can be done by taking the derivative of the criterion, setting it equal to zero, and solving for $\mathbf{c}$:

$$ D^1_\mathbf{c}[\mathbf{e}'\mathbf{e} + \lambda \mathbf{c}' P \mathbf{c} ] = 0$$

$$ D^1_\mathbf{c}[(\mathbf{y} - X\mathbf{c})'(\mathbf{y} - X\mathbf{c}) + \lambda \mathbf{c}' P \mathbf{c} ] = 0$$

$$ -2X'\mathbf{y} + 2X'X\mathbf{c} + 2 \lambda P \mathbf{c} = 0$$

We can divide both sides by two:

$$ -X'\mathbf{y} + X'X\mathbf{c} + \lambda P \mathbf{c}= 0$$

$$ X'X\mathbf{c} + \lambda P \mathbf{c} =  X'\mathbf{y}$$

$$ (X'X + \lambda P) \mathbf{c} =  X'\mathbf{y}$$

$$ \hat{\mathbf{c}} = (X'X + \lambda P)^{-1}X'\mathbf{y} $$

Let us now estimate $\mathbf{c}$ using the estimator we just derived, letting $\lambda=1$ to start, and predict the response in order to calculate the SSE:

In [ ]:
val lambda = 1.0
val cPen   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
val zPen   = X * cPen      // predicted response
val ePen   = y - zPen      // residual
val ssePen = ePen dot ePen // sum of squared error
::plotv y zPen
println(s"SSE = $ssePen")

Well, the fit not only looks better, it is better as evidenced by the smaller SSE. Can we make it even better? Consider the assumption we made of $\lambda=1$. Is there a different value that improves the SSE? Let's create a function that computes the SSE for a particular $\lambda$ value:

In [ ]:
def tryLambda(lambda: Double): Double = {
    val c   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
    val z   = X * c   // predicted response
    val e   = y - z   // residual
    e dot e           // return the sum of squared error
} // tryLambda

Now that we have a function that can be used for any $\lambda$, let's try repeatedly decreasing its value so long as it improves the SSE for the fit. While there are many different optimization techniques that could be used to find the optimal lambda my minimizing the objective function defined by `tryLambda`, we'll use a naive technique of repeated division for illustrative purposes:

In [ ]:
var lambda  = 1.0
var sse     = Double.PositiveInfinity
var ssePrev = Double.PositiveInfinity

do {
  lambda  = lambda / 10       // make lambda smaller
  ssePrev = sse               // save previous SSE
  sse     = tryLambda(lambda) // compute new SSE
  println(s"lambda = $lambda; sse = $sse")
} while (sse < ssePrev)       // loop while SSE improves

lambda = lambda * 10          // save the best SSE

Let's observe the fit using this new, optimal $\lambda$ value: 

In [ ]:
val cPen2   = ((X.t * X) + (P * lambda)).inverse * X.t * y // use roughness penalty
val zPen2   = X * cPen2       // predicted response
val ePen2   = y - zPen2       // residual
val ssePen2 = ePen2 dot ePen2 // sum of squared error
::plotv y zPen2
println(s"lambda = $lambda; sse = $ssePen2")

That's way better! The SSE is really good. We can now, finally and confidently, define $f(t)$ using the coefficient vector that we estimated for the smoothing spline:

In [ ]:
def f(t: Double) = (0 until bs.count()).map(i => c(i) * phi(i)(t)).sum 

Now you should be able to use $f(t)$ to compute values anywhere along closed interval defined by the ends of the original knot vector.

## References
* Patrikalakis & Maekawa (2009). *Shape Interrogation for Computer Aided Design and Manufacturing.* Springer Science & Business Media.
* Brockwell & Davis (1996). *Introduction to Time Series and Forecasting.* Springer.